## Exploring Rossmann Drug Store Sales Data

In [1]:
import pandas as pd
# pandas is specialized for time series data!

data = pd.read_csv('../../DS-SF-32/lessons/lesson-17/rossmann.csv', skipinitialspace=True,
                   low_memory=False)

IOError: File ../../DS-SF-32/lessons/lesson-17/rossmann.csv does not exist

In [ ]:
data.head(3)

Because we are most interested in the `Date` column that contains the date of sales for each store, we will make sure to process that as a `DateTime` type, and make that the index of our dataframe.

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
# using datetime type variables as an index is a huge strength
# and specialization of pandas
data.set_index('Date', inplace=True)

data['Year'] = data.index.year
data['Month'] = data.index.month

In [ ]:
data.head(3)

In [ ]:
store1_data = data[data.Store == 1]

In [ ]:
store1_data.head()

### Data Exploration

To compare sales on holidays, we can compare the sales using box-plots, which allows us to compare the distribution of sales on holidays against all other days. On state holidays the store is closed (and as a nice sanity check there are 0 sales), and on school holidays the sales are relatively similar.

In [ ]:
import seaborn as sb
%matplotlib inline

sb.factorplot(
    x='SchoolHoliday',
    y='Sales',
    data=store1_data, 
    kind='box'
)

In [ ]:
sb.factorplot(
    col='Open',
    x='DayOfWeek',
    y='Sales',
    data=store1_data,
    kind='box',
    
)
store1_data

### In class assignment:

+ Use the code above to explore if there are differences in sales on days with a promotion

In [ ]:
sb.factorplot(
    x='Promo',
    y='Sales',
    data=store1_data[store1_data['Open']==1],
    kind='box'
)

In [ ]:
sb.factorplot(
    col='Promo',
    x='DayOfWeek',
    y='Sales',
    data=store1_data[store1_data['Open']==1],
    kind='box'
)

Lastly, we want to identify larger-scale trends in our data. How did sales change from 2014 to 2015? Were there any particularly interesting outliers in terms of sales or customer visits?

In [ ]:
# Filter to days store 1 was open
store1_open_data = store1_data[store1_data.Open==1]
store1_open_data[['Sales']].plot()

In [ ]:
store1_open_data[['Customers']].plot()

In pandas we can compute rolling average using the `pd.rolling_mean` or `pd.rolling_median` functions.

### Data REFINING Using Time Series Statistics

### Autocorrelation

To measure how much the sales are correlated with each other, we want to compute the _autocorrelation_ of the 'Sales' column. In pandas, we do this we with the `autocorr` function.

`autocorr` takes one argument, the `lag` - which is how many prior data points should be used to compute the correlation. If we set the `lag` to 1, we compute the correlation between every point and the point directly preceding it, while setting `lag` to 10, computes the correlation between every point and the point 10 days earlier.

In [ ]:
# group data by day ('D')
# (requires indices that are datetimes)
daily_sales_data = data['Sales'].resample('D')
# get the mean of each day
daily_means = daily_sales_data.mean()
daily_means

In [ ]:
# find correlation with previous day (pearson R coefficient)
print daily_means.autocorr(lag=1)
print daily_means.autocorr(lag=7)

### Rolling Averages

If we want to investigate trends over time in sales, as always, we will start by computing simple aggregates.  We want to know what the mean and median sales were for each month and year.

In Pandas, this is performed using the `resample` command, which is very similar to the `groupby` command. It allows us to group over different time intervals.

We can use `data.resample` and provide as arguments:
    - The level on which to roll-up to, 'D' for day, 'W' for week, 'M' for month, 'A' for year
    - What aggregation to perform: 'mean', 'median', 'sum', etc.

In [ ]:
monthly_sales_data = data[['Sales']].resample('M')
monthly_sales_data.apply(['median', 'mean']).head()

While identifying the monthly averages are useful, we often want to compare the sales data of a date to a smaller window. To understand holidays sales, we don't want to compare late December with the entire month, but perhaps a few days surrounding it. We can do this using rolling averages.

In pandas, we can compute rolling average using the `df.rolling().mean()` or `df.rolling().median()` functions.

In [ ]:
daily_sales_means = data[['Sales']].resample('D').mean()
rolling_daily_sales_means = daily_sales_means.rolling(window=7, center=True)
rolling_daily_sales_means.mean().head(10)

`rolling()` takes these important parameters:
    - `window` is the number of days to include in the average
    - `center` is whether the window should be centered on the date or use data prior to that date
    - `freq` is on what level to roll-up the averages to (as used in `resample`). Either `D` for day, `M` for month or `A` for year, etc.

Instead of plotting the full timeseries, we can plot the rolling mean instead, which smooths random changes in sales as well as removing outliers, helping us identify larger trends.

In [ ]:
data[['Sales']].resample('D').mean().plot()

In [ ]:
data[['Sales']].resample('D').mean().rolling(window=10, center=True).mean().plot()

### Pandas Window functions
Pandas `rolling_mean` and `rolling_median` are only two examples of Pandas window function capabilities. Window functions are operate on a set of N consecutive rows (a window) and produce an output.

In addition to `rolling_mean` and `rolling_median`, there are `rolling_sum`, `rolling_min`, `rolling_max`... and many more.

Another common one is `diff`, which takes the difference over time. `pd.diff` takes one arugment, `periods`, which is how many prior rows to use for the difference.


In [ ]:
data['Sales'].diff(periods=1).head()

### Pandas expanding functions

In addition to the set of `rolling_*` functions, Pandas also provides a similar collection of `expanding_*` functions, which, instead of using a window of N values, use all values up until that time.

In [ ]:
# computes the average sales, from the first date _until_ the date specified.
data[['Sales']].resample('D').mean().expanding().mean().head()


## Exercises

In [ ]:
sb.factorplot(
    col='Open',
    hue='Promo',
    x='Month',
    y='Sales',
    data=store1_data, 
    kind='box'
)

### In class assignment

> Modify code above to Plot the distribution of sales by month and compare the effect of promotions

In [ ]:
sb.factorplot(
    col='Promo',
    hue='Open',
    x='Month',
    y='Sales',
    data=store1_data, 
    kind='box'
)

> Are sales more correlated with the prior date, a similar date last year, or a similar date last month?

In [ ]:
average_daily_sales = data[['Sales', 'Open']].resample('D').mean()
average_monthly_sales = data[['Sales', 'Open']].resample('M').mean()

print('Correlation with last day: {}'.format(average_daily_sales['Sales'].autocorr(lag=1)))
print('Correlation with this day "last month:" {}'.format(average_daily_sales['Sales'].autocorr(lag=30)))
print('Correlation with last month (by month): {}'.format(average_monthly_sales['Sales'].autocorr(lag=1)))
print('Correlation with this day "last year:" {}'.format(average_daily_sales['Sales'].autocorr(lag=365)))

> Plot the 15 day rolling mean of customers in the stores

In [ ]:
# larger window smooths out effects even more
data[['Sales']].resample('D').mean().rolling(window=15).mean().plot()

> Identify the date with largest drop in sales from the same date in the previous week

In [ ]:
average_daily_sales = data[['Sales', 'Open']].resample('D').mean()
average_daily_sales['DiffVsLastWeek'] = average_daily_sales[['Sales']].diff(periods=7)

average_daily_sales.sort_values(by='DiffVsLastWeek').head()

# Unsurprisingly, this day is Dec. 25 and Dec. 26 in 2014 and 2015. When the store is closed and there are many sales in the preceding week. How, about when the store is open?

average_daily_sales[average_daily_sales.Open == 1].sort_values(by='DiffVsLastWeek').head()

# The top values are Dec. 24 and then 2013-12-09 and 2013-10-14 where on average sales were 4k lower than the same day in the previous week.


> Compute the total sales up until Dec. 2014

In [ ]:
total_daily_sales = data[['Sales']].resample('D').sum()

total_daily_sales.expanding().sum()['2014-12'].head()


> When were the largest differences between 15-day moving/rolling averages?
> HINT: Using `rolling(...).mean()` and `diff`

In [ ]:
daily_sales_means = data[['Sales']].resample('d').mean()
rolling_15day_means = daily_sales_means.rolling(window=15, center=False).mean()
oneday_diffs_in_15day_means = rolling_15day_means.diff(1)
oneday_diffs_in_15day_means.sort_values(by='Sales').tail(20)

# Unsurprisingly, they occur at the beginning of every year after the holiday season.